# <B> Clean up </B>

## AutoReload

In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## clean

In [60]:
import os
import time
import boto3
from utils.ssm import parameter_store

In [61]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

* S3 Bucket

In [62]:
strBucketName = pm.get_params(key=strPrefix + "BUCKET")

In [63]:
strQuery = "".join(["aws s3 rm --recursive ", '"s3://', strBucketName, '"'])
strResponse = os.popen(strQuery).read()
strQuery = "".join(["aws s3 rb ", '"s3://', strBucketName, '"'])
strResponse = os.popen(strQuery).read()

* endpoint

In [64]:
class clean_up():
    
    def __init__(self, ):    
        pass
    
    def delete_endpoint(self, client, endpoint_name ,is_del_model=True):
        
        response = client.describe_endpoint(EndpointName=endpoint_name)
        EndpointConfigName = response['EndpointConfigName']

        response = client.describe_endpoint_config(EndpointConfigName=EndpointConfigName)
        model_name = response['ProductionVariants'][0]['ModelName']    

        if is_del_model: # 모델도 삭제 여부 임.
            client.delete_model(ModelName=model_name)    

        client.delete_endpoint(EndpointName=endpoint_name)
        client.delete_endpoint_config(EndpointConfigName=EndpointConfigName)    

        print(f'--- Deleted model: {model_name}')
        print(f'--- Deleted endpoint: {endpoint_name}')
        print(f'--- Deleted endpoint_config: {EndpointConfigName}')  

In [65]:
clean = clean_up()
sm_client = boto3.client('sagemaker')
strEndPointName = pm.get_params(key=strPrefix + "ENDPOINT-NAME")
clean.delete_endpoint(sm_client, strEndPointName ,is_del_model=True)

--- Deleted model: pytorch-inference-2023-03-03-10-42-49-298
--- Deleted endpoint: endpoint-model-2303041677839910
--- Deleted endpoint_config: endpoint-model-2303041677839910


* pipeline

In [66]:
strPipelineName = pm.get_params(key=strPrefix + "PIPELINE-NAME")

In [67]:
!aws sagemaker delete-pipeline --pipeline-name $strPipelineName 

{
    "PipelineArn": "arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/sm-byom-pipeline-train-sample-model-230304"
}


* model registry

In [68]:
sagemaker_client = boto3.client('sagemaker')

In [69]:
def empty_and_delete_model_package(sagemaker_client, mpg_name):
    mpg = sagemaker_client.list_model_packages(
        ModelPackageGroupName=mpg_name,
    )
    
    # Delete model packages if Group not empty
    model_packages = mpg.get('ModelPackageSummaryList')
    if model_packages:
        for mp in model_packages:
            sagemaker_client.delete_model_package(
                ModelPackageName=mp['ModelPackageArn']
            )
            time.sleep(1)

    # Delete model package group
    sagemaker_client.delete_model_package_group(
        ModelPackageGroupName=mpg_name
    )

In [70]:
DeleteModelGroupName = pm.get_params(key=strPrefix + "MODEL-GROUP-NAME")
#DeleteModelGroupName = "model-24"
empty_and_delete_model_package(sagemaker_client, DeleteModelGroupName)

* parameter store

In [71]:
listParams = pm.get_all_params()
listParams = [strParam for strParam in listParams if strPrefix in strParam] + ["PREFIX"]
pm.delete_param(listParams)

  parameters: ['SM-BYOM-ACCOUNT-ID', 'SM-BYOM-BUCKET', 'SM-BYOM-DATA-PATH', 'SM-BYOM-ENDPOINT-NAME', 'SM-BYOM-LAMBDA-ROLE-ARN', 'SM-BYOM-MODEL-GROUP-NAME', 'SM-BYOM-PIPELINE-NAME', 'SM-BYOM-REGION', 'SM-BYOM-SAGEMAKER-ROLE-ARN', 'PREFIX'] is deleted successfully
